<a href="https://colab.research.google.com/github/pauloigorms/covid-twitter-pecd/blob/main/Programacao_Ciencia_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PROGRAMAÇÃO PARA CIÊNCIA DE DADOS**

**Equipe**: Cássio de Paula / Paulo Moraes [(website)](https://paulomoraes.me).

**Objetivo**: analisar as mensagens
trocadas por usuários do Twitter sobre a COVID-19, com o intuito de realizar uma análise
exploratória e visualização de dados, passando também pelas etapas de limpeza e organização.
Mais especificamente, as equipes devem explorar a análise nas perguntas (questões) dos usuários,
pois arguimos que seja um tipo de mensagem apropriado para se compreender as principais dúvidas
das pessoas sobre a atual pandemia.


# **INFORMAÇÕES GERAIS**

### **ARTEFATOS**


*   Relatório Técnico - *Um notebook com o texto e o código-fonte apresentado na ferramenta Google COLAB*; 
*   Repositório no GitHub.


### **ATIVIDADES**


*   Importar base de dados;
*   Tratar base de dados;
*   Identificar as perguntas (questões) descritas nas mensagens;
*   Salvar coleção com a identificação das perguntas em um novo dataset chamado "DuvidasDB".


### **DO RESULTADO**


1.   Devem apresentar um resumo (sumário) com as estatísticas dos dados originais, elas podem ser apresentadas através de tabelas e/ou gráficos;
2.   Com a coleção resultando da identificação das perguntas a equipe deve apresentar as estatísticas sobre estes dados.


### **TEMAS DISCUTIDOS**

As equipes devem fazer uma análise sobre os temas que são debatidos nas perguntas dos postadas pelos usuários. Faça uma análise se as perguntas são relativas aos seguintes temas:

*   Doença. Quando a pergunta é relativa à doença Deve-se observar que a doença é identificada por vários nomes. Exemplo: coronavírus, corona, COVID-19, etc;
*   Medicamento. Quando a pergunta é sobre o uso de determinado medicamento no tratamento da doença;
*   Organizações. Quando a pergunta é relativa a uma determinada entidade ou organização. Emissora de TV, Ministério da Saúde ou empresas são exemplos de organizações;
*   Pessoas. Quando a pergunta é sobre determinada pessoa. Por exemplo, a pergunta pode ser sobre a atuação que determinado político ou pessoa famosa teve durante esse período de pandemia.

# **VISÃO DOS DADOS**

> **Visão Temporal** - Deve-se fazer uma análise temporal das perguntas que formam o dataset *DuvidasDB*. Pode-se considerar o intervalo temporal de dias, semanas ou meses. A escolha do intervalo de tempo ficará a cargo das equipes. Exemplos de análise temporal: a) houve um aumento no número de perguntas
ao longo do tempo? b) houve uma mudança no perfil das perguntas ao longo do tempo?

> **Visão Geográfica** - Deve-se fazer uma análise geográfica (espacial) das perguntas que formam o dataset *DuvidasDB*. Existem algumas colunas no dataset que trazem a informação das localizações como, por exemplo, o país, estado e cidade. Em alguns tuítes é possível ainda identificar as coordenadas geográficas
de latitude e longitude. Exemplo de análise geográfica: a) os usuários de regiões diferentes fazem
perguntas com diferentes focos? Por exemplo, será que os usuários de uma região perguntam mais
sobre a doença ou sobre o tratamento? Essa análise ainda pode ser realizada em diversos níveis de
área (cidade, estado ou região). Além de apresentar a distribuição das dúvidas dos usuários por
região, a equipe deverá fazer uma análise dessa distribuição. Por exemplo, apresentar as razões
(ou hipóteses) da ocorrência dessa distribuição.


# **INICIANDO O PROJETO**

IMPORTANDO LIBS

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import nltk as nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


LEITURA DO ARQUIVO DE DADOS BASE

In [ ]:
drive.mount('/content/drive')
load_data = pd.read_csv('drive/My Drive/twitter/dados-curso-completo.csv', sep= ',',skiprows=0, encoding='utf-8')
raw_data = load_data.drop_duplicates(['texto'])

Mounted at /content/drive


#Funções

Função: **Cálcular Percente**

In [ ]:
def calc_percente(value, total): 
    if not value and not total: 
       return print("value = 0% \n total = 0%") 
    elif value < 0 or total < 0: 
       return print("não aceita valor negativo!") 
    else: 
       return print('{0:.2f}%'.format((value / total * 100)))

Função: **Formatar String**

In [85]:
def spliting(text):
  return text.split()

#Estatísticas bases


**Exemplo da base de dados**

---



In [123]:
raw_data.head(50)
# nodp_data.info()

,data,usuario,apelido,texto,retweet,seguidores,idioma,lugar,pais,sigla,latitude,longitude,hashtags
data,,,,,,,,,,,,,
2020-03-23 18:51:14,2020-03-23 18:51:14,Gaboardi,fraslee,@BolsonaroSP Cade o fundão deputado??? Congres...,0,138,pt,NaN,NaN,NaN,NaN,NaN,[]
2020-03-18 03:18:14,2020-03-18 03:18:14,Seu Zé da Farmácia,seuzedafarmacia,"Em ação preventiva, o Governo do Estado suspen...",0,56,pt,NaN,NaN,NaN,NaN,NaN,[]
2020-03-29 12:09:27,2020-03-29 12:09:27,Marília Oliveira 🇧🇷,Marioliveira_gb,Incrível como os defensores do fim do distanci...,0,5,pt,NaN,NaN,NaN,NaN,NaN,[]
2020-04-01 21:11:07,2020-04-01 21:11:07,🚩,santosodeto,Fumaça de cigarro pode propagar coronavírus? h...,0,1885,pt,NaN,NaN,NaN,NaN,NaN,[]
2020-03-19 17:49:40,2020-03-19 17:49:40,Nova Impressão,novaimpressao_,🚨Todos juntos contra COVID-19💪\n\nSolicite seu...,0,124,pt,Brasília,Brazil,BR,-47.8778,-15.77691,[]
2020-05-08 11:21:36,2020-05-08 11:21:36,Marcel Silvano,marcelsilvano,"Importantes avanços! A pesquisa brasileira, a ...",4,2769,pt,Macaé,Brazil,BR,NaN,NaN,[]
2020-03-10 21:27:40,2020-03-10 21:27:40,Teixeira Correia,LidadorNoticias,(Última hora) Volta ao Alentejo: Anulada por c...,0,39,pt,NaN,NaN,NaN,NaN,NaN,[]
2020-06-04 13:22:21,2020-06-04 13:22:21,Thiago Moraes,ThiagoMoraesPB,João Pessoa tem queda no isolamento social por...,1,3578,pt,NaN,NaN,NaN,NaN,NaN,[]
2020-05-07 21:32:12,2020-05-07 21:32:12,PT Brasil,ptbrasil,A #tvPT de hoje vai falar sobre a pandemia na ...,10,942696,pt,NaN,NaN,NaN,NaN,NaN,['tvPT']


**Tamanha da base (com e sem dados duplicados)**

---

In [ ]:
# Total com duplicados: 1.658.825
# Total sem duplicados = 1.653.599
raw_data.texto.count()

1653599

**Twitters por região**

---

In [120]:
# Total de twitters com localização = 74.383
raw_data['lugar'].count()

74383

In [ ]:
# Percentual em relação à base = 4.50%
calc_percente(value=raw_data['lugar'].count(), total=raw_data.texto.count())

4.50%


In [121]:
# Total por país
filter = raw_data[raw_data.pais == 'Brazil'].groupby('pais').count()
filter.head()
# Total por estado
# raw_data[raw_data.pais == 'Brazil'].groupby(['pais', 'lugar']).count() #.head(2)

,data,usuario,apelido,texto,retweet,seguidores,idioma,lugar,sigla,latitude,longitude,hashtags
pais,,,,,,,,,,,,
Brazil,67034,67033,67034,67034,67034,67034,67034,67034,67034,7675,7675,67034


**Twitters por período**

---

In [125]:
df_data = pd.DataFrame(raw_data)
df_data['data'] = pd.to_datetime(df_data['data'])
df_data.index = df_data['data']
df_data.resample('M').count()

,data,usuario,apelido,texto,retweet,seguidores,idioma,lugar,pais,sigla,latitude,longitude,hashtags
data,,,,,,,,,,,,,
2020-01-31,54391,54388,54391,54391,54391,54391,54391,2177,2177,2177,80,80,54391
2020-02-29,109489,109485,109489,109489,109489,109489,109489,4305,4305,4305,225,225,109489
2020-03-31,480128,480112,480128,480128,480128,480128,480128,23487,23486,23484,2561,2561,480128
2020-04-30,469334,469321,469334,469334,469334,469334,469334,21318,21317,21315,2516,2516,469334
2020-05-31,283229,283219,283229,283229,283229,283229,283229,12780,12777,12777,1993,1993,283229
2020-06-30,257027,257022,257027,257027,257027,257027,257027,10316,10316,10315,1345,1345,257027
2020-07-31,1,1,1,1,1,1,1,0,0,0,0,0,1


**Mineração de Perguntas**

---






*Tamanho da coluna que precisa ser minerada*

In [ ]:
valor = len(raw_data['texto'])
valor

1653599

*Amostra de checagem para suspeita de linhas com erros*

In [ ]:
sentencas = sent_tokenize(raw_data['texto'][7159]) # this gives us a list of sentences
sentencas

['@TeichNelson @minsaude @Medicina_CFM @Astro_Pontes Ajudar a enxergar coisas no raio-x, que indicaram algo como o vi… https://t.co/zPbNfdIHll']

*Mineração para Recuperação de Perguntas na Base*

In [ ]:
twitter = []
perguntas = []
erro = []
for i in range(0,valor):
  try:
    twitter.append(sent_tokenize(raw_data['texto'][i]))   
    for sentencas in twitter: 
      for row in sentencas:
        palavras = word_tokenize(row.lower())
        for palavra in palavras:
          if palavra == '?':
            perguntas.append(row) 
            break
    twitter.clear()
  except:
    erro.append(i)     

 

*Resumo após a Mineração*

In [ ]:
len(erro)
#Quantidade de registros que quebrariam a mineração

0

In [ ]:
len(perguntas)
#Quantidade de perguntas extraidas

127

**DataSet gerado para a análise**

In [ ]:
DuvidasDB = perguntas
len(DuvidasDB)

2

In [ ]:
perguntas

['@BolsonaroSP Cade o fundão deputado???',
 'Fumaça de cigarro pode propagar coronavírus?',
 '@ipratesMD @dr_pampers Será que corremos o risco de termos piolhos super resistentes?',
 'Prioridade ?!',
 'Que tal uma notícia boa, indo na contramão do que a mídia corrupta está fazendo (tocando o terror na população)?',
 '@JoseMedeirosMT Seria isso algum tipo aleatório de demência provocando pelo coronavírus?',
 '"Nova York é o novo epicentro da pandemia de corona vírus"\n\nTa agora eu lhe pergunto cadê os vingadores?',
 '@mitags Será por que o Bolsonaro não fez propaganda?',
 '@JeanSCorrea7 Mas ele não deu a entender que o Tamiflu era melhor, neh?',
 '#perguntacorona \nQuero saber se a covid tem cura?',
 '@elmundoes eres republicano?',
 'E essa chuvinha boa para pegar um resfriado e confundir com Covid-19, hem?',
 'Quem é você na Pandemia?',
 'Cadê aquela polícia do Dória que prendia velhos, mulheres e trabalhadores na pandemia ???',
 '@paulomathias Pergunta pra ele pq só a Cracolândia est

*Rascunho geral*

In [ ]:
from nltk.corpus import stopwords
from string import punctuation
stopwords = set(stopwords.words('portuguese') + list(punctuation))
stopwords
#list(punctuation)

palavras_sem_stopwords = [palavra for palavra in sentencas if palavra not in stopwords]
palavras_sem_stopwords